<a href="https://colab.research.google.com/github/jaayg23/workshop_PDI/blob/main/proyecto_PDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
from ultralytics import YOLO

model = YOLO('yolo11n')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 99.4MB/s]


In [4]:
!unzip "/content/drive/MyDrive/Colab Notebooks/PDI/VisDrone2019-VID-val.zip" -d /content/data

Archive:  /content/drive/MyDrive/Colab Notebooks/PDI/VisDrone2019-VID-val.zip
   creating: /content/data/VisDrone2019-VID-val/
   creating: /content/data/VisDrone2019-VID-val/annotations/
  inflating: /content/data/VisDrone2019-VID-val/annotations/uav0000086_00000_v.txt  
  inflating: /content/data/VisDrone2019-VID-val/annotations/uav0000117_02622_v.txt  
  inflating: /content/data/VisDrone2019-VID-val/annotations/uav0000137_00458_v.txt  
  inflating: /content/data/VisDrone2019-VID-val/annotations/uav0000182_00000_v.txt  
  inflating: /content/data/VisDrone2019-VID-val/annotations/uav0000268_05773_v.txt  
  inflating: /content/data/VisDrone2019-VID-val/annotations/uav0000305_00000_v.txt  
  inflating: /content/data/VisDrone2019-VID-val/annotations/uav0000339_00001_v.txt  
   creating: /content/data/VisDrone2019-VID-val/sequences/
   creating: /content/data/VisDrone2019-VID-val/sequences/uav0000086_00000_v/
  inflating: /content/data/VisDrone2019-VID-val/sequences/uav0000086_00000_v/000

In [7]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

def signed_distance(point, line):
    x, y = point
    (x1, y1), (x2, y2) = line
    num = (y2 - y1) * x - (x2 - x1) * y + x2 * y1 - y2 * x1
    den = np.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)
    return num / den

count_line1 = 0
count_line2 = 0
distance_threshold = 25
prev_centroids = []

# Rutas (ajusta estas rutas)
image_folder = '/content/data/VisDrone2019-VID-val/sequences/uav0000086_00000_v'  # Carpeta con imágenes
output_path = 'output.avi'

# Cargar modelo YOLO
model = YOLO('yolo11n.pt')

# Obtener lista de imágenes ordenadas
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))])
if not image_files:
    raise ValueError("No se encontraron imágenes en la carpeta.")

# Leer primera imagen para obtener tamaño
first_frame = cv2.imread(os.path.join(image_folder, image_files[0]))
height, width = first_frame.shape[:2]
fps = 30  # Asume un FPS constante o ajústalo al valor real

# Configurar VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Procesar cada imagen como un frame
for filename in image_files:
    frame_path = os.path.join(image_folder, filename)
    frame = cv2.imread(frame_path)
    if frame is None:
        continue

    results = model(frame, conf=0.3)
    boxes_obj = results[0].boxes

    current_centroids = []
    if boxes_obj is not None and len(boxes_obj) > 0:
        bboxes = boxes_obj.xyxy.cpu().numpy()
        classes = boxes_obj.cls.cpu().numpy()

        for i in range(len(bboxes)):
            if int(classes[i]) == 0:
                x1, y1, x2, y2 = map(int, bboxes[i])
                centroid = ((x1 + x2) // 2, (y1 + y2) // 2)
                current_centroids.append(centroid)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.circle(frame, centroid, 4, (0, 255, 0), -1)


    writer.write(frame)
    prev_centroids = current_centroids.copy()

writer.release()
print(f"Video procesado guardado en: {output_path}")



0: 384x640 14 persons, 1 sports ball, 11.8ms
Speed: 2.6ms preprocess, 11.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 1 sports ball, 9.1ms
Speed: 2.8ms preprocess, 9.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 sports ball, 13.4ms
Speed: 3.1ms preprocess, 13.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 sports ball, 9.9ms
Speed: 3.3ms preprocess, 9.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 sports ball, 9.3ms
Speed: 3.0ms preprocess, 9.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 1 sports ball, 9.5ms
Speed: 2.8ms preprocess, 9.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 sports ball, 12.1ms
Speed: 3.2ms preprocess, 12.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384